In [ ]:
import csv
import json
import os
import time

import gym
import roboschool

import numpy as np

from gym import wrappers
from ipywidgets import Video
from multiprocessing import Pool


## Load a Model

Since we use a custom initializer and this gets serialized during the saving process of the model we need to pass it on when we load it again. Unfortunately with the issue of the background TensorFlow session when importing TensorFlow and multiprocessing we cannot define the initializer one time and use it here again. So we define it twice, one inside the create_model() function and here.

In [ ]:
def load_model(model_path):   
    import tensorflow as tf

    class Normc_initializer(tf.keras.initializers.Initializer):
        """
        Create a TensorFlow constant with random numbers normed in the given shape.
        :param std:
        :return:
        """
        def __init__(self, std=1.0):
            self.std = std

        def __call__(self, shape, dtype=None, partition_info=None):
            out = np.random.randn(*shape).astype(np.float32)
            out *= self.std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
            return tf.constant(out)
        
    class ObservationNormalizationLayer(tf.keras.layers.Layer):
        def __init__(self, ob_mean, ob_std, **kwargs):
            self.ob_mean = ob_mean
            self.ob_std = ob_std
            super(ObservationNormalizationLayer, self).__init__(**kwargs)

        def call(self, input):
            return tf.clip_by_value((input - self.ob_mean) / self.ob_std, -5.0, 5.0)
          
        def get_config(self):
            base_config = super(ObservationNormalizationLayer, self).get_config()
            base_config['ob_mean'] = self.ob_mean
            base_config['ob_std'] = self.ob_std
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
    
    custom_objects = {'Normc_initializer' : Normc_initializer(std=1.0), 
                      'ObservationNormalizationLayer' : ObservationNormalizationLayer}
    
    return tf.keras.models.load_model(model_path, custom_objects=custom_objects)

# Visualize

In [ ]:
def rollout(env, model, render=False, timestep_limit=None, random_stream=None):
    """
    If random_stream is provided, the rollout will take noisy actions with noise drawn from that stream.
    Otherwise, no action noise will be added.
    """

    env_timestep_limit = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    timestep_limit = env_timestep_limit if timestep_limit is None else min(timestep_limit, env_timestep_limit)
    rews = []
    t = 0

    ob = env.reset()
    for _ in range(timestep_limit):
        if render:
            env.render()
        ac = act(ob[None], model, random_stream=random_stream)[0]
        ob, rew, done, _ = env.step(ac)
        rews.append(rew)
        t += 1

        if done:
            break
    return np.array(rews, dtype=np.float32), t

def act(ob, model, random_stream=None):   
    action = model.predict(ob)
    
    #if random_stream is not None and model_structure.ac_noise_std != 0:
    #    action += random_stream.randn(*action.shape) * model_structure.ac_noise_std
    
    return action

In [ ]:
def _visualize_in_process(env_id, model_file, save_directory, record=False):
    env = gym.make(env_id)
    env.reset()
    
    video_directory = save_directory + '/videos/' + model_file + '/'
    
    if record:
        env = wrappers.Monitor(env, video_directory, force=True)
        
    model = load_model(save_directory + model_file)
    
    rollout(env, model)
    
    return video_directory

def visualize(save_directory):
    files = [save_directory + file for file in os.listdir(save_directory)]
    
    model_files, config_files = [], {}
    
    for file in os.listdir(save_directory):
        if file.endswith('.h5'):
            model_files.append(file)
        elif file.endswith('.json'):
            with open(save_directory + file, 'r') as f:
                config_files = json.load(f)
    
    model_files.sort()

    video_directories = []

    with Pool(os.cpu_count()) as pool:
        for m in model_files:
            video_directories.append(pool.apply_async(func=_visualize_in_process, args=(config_files['config']['env_id'], 
                                                        m,
                                                        save_directory,
                                                        True)))
        
        for i in range(len(video_directories)):
            video_directories[i] = video_directories[i].get()
    
    return video_directories        

In [ ]:
save_directory = '/tmp/es_2702/'

video_directories = visualize(save_directory)

In [ ]:
videos = []
for directory in video_directories:
    for file in os.listdir(directory):
        if file.endswith('.mp4'):
            videos.append(Video.from_file(directory + file))
            
for video in videos:
    video

    

In [ ]:
def parse_log_to_csv(log_file, csv_file):
    with open(log_file) as f:
        content = f.readlines()

    groups = temp =  []
    for line in content:
        line = line.split()

        if not line:
            continue

        if "Generation" in line:
            temp = [line[-1]]
            groups.append(temp)
        else:
            temp.append(line[-1])

    writer = csv.writer(open(csv_file, 'w'))

    writer.writerow(['Generation',
                     'Reward Mean',
                     'Reward Standard Deviation',
                     'Length Mean',
                     'Evaluation Reward Mean',
                     'Evaluation Reward Standard Deviation',
                     'Evaluation Length Mean',
                     'Evaluation Count',
                     'Episodes this generation',
                     'Episodes overall',
                     'Timesteps this generation',
                     'Timesteps overall',
                     'Unique Workers',
                     'Observation count',
                     'Time elapsed this generation (s)',
                     'Time elapsed overall (s)'])

    for generation in groups:
        if len(generation) != 18: continue

        row = []

        for column in generation:
            row.append(column)

        writer.writerow(row)
